In [ ]:
import sys
from pathlib import Path

# The correct way to get the project root.
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root added to path: {project_root}")

In [ ]:
# block 1

import pandas as pd
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from emoji_sentiment_analysis.config import MODELS_DIR

# Regex to find emojis
EMOJI_PATTERN = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # Emoticons
    "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
    "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
    "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
    "\U00002702-\U000027B0"  # Dingbats
    "]+",
    flags=re.UNICODE
)

def extract_emojis(text):
    return " ".join(re.findall(EMOJI_PATTERN, text))

# Load the trained model and vectorizer
model = joblib.load(MODELS_DIR / "sentiment_model.pkl")
vectorizer = joblib.load(MODELS_DIR / "tfidf_vectorizer.pkl")

print("Sentiment analyzer loaded successfully! 🚀")

In [ ]:
# block 2

# --- Widgets ---
text_input = widgets.Textarea(
    description='Enter a sentence:',
    placeholder='I love sentiment analysis but it is too hard to study 😔',
    layout=widgets.Layout(width='100%', height='100px')
)
sentiment_button = widgets.Button(
    description='Say your Sentiments!',
    button_style='success'
)
output_area = widgets.Output()

display(text_input, sentiment_button, output_area)

# --- Helpers ---
def preprocess_text(s: str) -> str:
    s = (s or "").strip()
    emojis = extract_emojis(s)  # from Block 1
    # append extracted emojis so the vectorizer can see them as tokens too
    return f"{s} {emojis}".strip()

# If this cell is re-run, remove any previous handler to avoid duplicate outputs
if 'sentiment_click_handler' in globals():
    try:
        sentiment_button.on_click(sentiment_click_handler, remove=True)
    except Exception:
        pass

# --- Callback ---
def on_button_click(b):
    with output_area:
        clear_output()
        input_text = text_input.value

        if not input_text or not input_text.strip():
            print("Please enter some text.")
            return

        prepared = preprocess_text(input_text)
        text_vectorized = vectorizer.transform([prepared])  # uses vectorizer from Block 1

        # Predict label
        prediction = model.predict(text_vectorized)[0]  # 0 or 1

        # Optional confidence (works if your classifier implements predict_proba)
        try:
            proba = float(model.predict_proba(text_vectorized)[0].max())
        except Exception:
            proba = None

        # Map numeric prediction to string label
        sentiment_labels = {0: "NEGATIVE", 1: "POSITIVE"}
        sentiment_label = sentiment_labels.get(int(prediction), "UNKNOWN")

        # --- Output ---
        print("==========================================================")
        print(f'YOUR INPUT IS OF "{sentiment_label}" SENTIMENT')
        if proba is not None:
            print(f"Confidence: {proba:.2%}")
        print("----------------------------------------------------------")
        print(f'Your input was: "{input_text.strip()}"')
        print("==========================================================")

# Register (and remember) the handler so we can remove it on re-run
sentiment_click_handler = on_button_click
sentiment_button.on_click(sentiment_click_handler)


In [ ]:
# block 3

def on_button_click(b):
    with output_area:
        clear_output()  # Clear previous output
        user_input = text_input.value
        
        if not user_input:
            print("Please enter a sentence to analyze.")
            return

        print("==========================================================")
        print(f'Your input is: "{user_input}"')

        # Preprocess the user input, including amplifying emojis
        processed_input = user_input + ' ' + extract_emojis(user_input) * 5
        
        # Transform the text using the trained vectorizer
        transformed_input = vectorizer.transform([processed_input])
        
        # Predict the sentiment
        prediction = model.predict(transformed_input)[0]
        
        # Display the result
        print("==========================================================")
        print(f'YOUR INPUT IS OF "{prediction.upper()}" SENTIMENT')
        print("==========================================================")

# Link the button's on_click event to the analysis function
sentiment_button.on_click(on_button_click)